In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import json
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.applications import EfficientNetB5, EfficientNetB3, EfficientNetB4
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation,RandomZoom
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras import layers
import json
from sklearn.model_selection import train_test_split
import os
import glob

In [ ]:
# Constants to access inputs
INPUT_PATH = "../input/cassava-leaf-disease-classification/"
MYFILES_PATH = "../input/trainedmodels/"
EFFICIENTNET_PATH = "../input/efficientnetb4/efficientnetb4_notop.h5"
TEST_IMAGES_PATH = INPUT_PATH + 'test_images/'
TRAINING_IMAGES_PATH = INPUT_PATH + '/train_images/'
TRAINING_LABEL_PATH = INPUT_PATH + 'train.csv'
IMAGE_SIZE = {
    'height':512,
    'width':512,
    'channels':3
}
LABEL_MAPPING_FILE_PATH = INPUT_PATH+'label_num_to_disease_map.json'
BATCH_SIZE = 128
NUM_CLASSES = 5
MODEL_NAME = "modelB5Noisy2.15-0.902.h5"
MODEL_WEIGHTS_NAME = "weights_modelB5Noisy2.15-0.902.h5"
#NUM_EPOCHS=24

In [ ]:
label_mapping = json.load(open(LABEL_MAPPING_FILE_PATH))
df_train = pd.read_csv(TRAINING_LABEL_PATH)
df_train['label_description'] = df_train.apply(lambda x : label_mapping.get(str(x['label']),"NA"),axis=1)
df_train = df_train.astype({'label': 'str'})

In [ ]:
def build_train_test_dataframes(df,test_size,stratified:bool):
    
    if stratified:
        strat = df.loc[:,['label']]
    else:
        strat = None
        
    return train_test_split(df.loc[:,['image_id','label','label_description']],df.loc[:,['label']],test_size = test_size,stratify = strat,random_state=94)

In [ ]:
X_train_df,X_validation_df,y_train,y_validation = build_train_test_dataframes(df_train,0.1,True)

In [ ]:
data_generator = ImageDataGenerator()

In [ ]:
train_generator = data_generator.flow_from_dataframe(dataframe=X_train_df,x_col="image_id",y_col="label",classes=["0","1","2","3","4"],
                                                    directory = TRAINING_IMAGES_PATH,target_size=(IMAGE_SIZE["height"],IMAGE_SIZE["width"]),
                                                    color_mode="rgb",class_mode="categorical",batch_size=BATCH_SIZE,validation_filenames=False,shuffle=False)


In [ ]:
validation_generator = data_generator.flow_from_dataframe(dataframe=X_validation_df,x_col="image_id",y_col='label',classes=["0","1","2","3","4"],
                                                    directory = TRAINING_IMAGES_PATH,target_size=(IMAGE_SIZE["height"],IMAGE_SIZE["width"]),
                                                    color_mode="rgb",class_mode="categorical",batch_size=BATCH_SIZE,validation_filenames=False,shuffle=False)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALIDATION=validation_generator.n//validation_generator.batch_size

In [ ]:
def load_model(model_path):
    model = tf.keras.models.load_model(model_path,compile=False)    return model

In [ ]:
def build_model():
    
    data_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.1),
        #preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomZoom((-0.2, 0.2)),
        preprocessing.RandomContrast(factor=0.2),
    ],name="data_augmentation",)
    
    inputs = layers.Input(shape=(IMAGE_SIZE["height"],IMAGE_SIZE["width"], 3))
    x = data_augmentation(inputs)
    x = layers.experimental.preprocessing.Rescaling(1./255)(x)
    backbone_model = EfficientNetB4(include_top=False,weights=None,input_tensor=x,drop_connect_rate=0.4)
    x = backbone_model(x, training=False)
    
    #We freeze the pretrained weights
    #backbone_model.trainable = False
    
    x = layers.GlobalAveragePooling2D(name="avg_pool")(backbone_model.output)
    x = layers.BatchNormalization(name="BN_out1",trainable=False)(x)
    #x = layers.Dense(768,activation='relu',name="FC_out1")(x)
    x = layers.Dropout(0.4,name="dropout_layer")(x)
    outputs = layers.Dense(NUM_CLASSES,activation="softmax",name="output_prediction")(x)
    
    model = Model(inputs, outputs,name="CassavaAugmentedEfficientNetB4TrainedOnNoisy")
    
    initial_learning_rate = 1e-4
    #decay_steps = int(STEP_SIZE_TRAIN)*NUM_EPOCHS
    #cosine_decay = CosineDecay(initial_learning_rate=initial_learning_rate,decay_steps=decay_steps, alpha=0.1)
    #lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #                initial_learning_rate,
    #                decay_steps=600,
    #                decay_rate=0.96,
    #                staircase=True)

    optimizer = tf.keras.optimizers.Adam()

    model.compile(
        
        optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3),
        metrics=["categorical_accuracy"]
    )
    
    return model

In [ ]:
model = build_model()

In [ ]:
model.load_weights(MYFILES_PATH+MODEL_WEIGHTS_NAME)

In [ ]:
model.summary()

In [ ]:
#def finetune_model():
    
#    initial_learning_rate = 1e-7
    #decay_steps = int(STEP_SIZE_TRAIN)*NUM_EPOCHS
    #cosine_decay = CosineDecay(initial_learning_rate=initial_learning_rate,decay_steps=decay_steps, alpha=0.3)
    #lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #                initial_learning_rate,
    #                decay_steps=600,
    #                decay_rate=0.96,
    #                staircase=True)

#    optimizer = tf.keras.optimizers.Adam()

#    model.compile(

#        optimizer=optimizer,
#        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3),
#        metrics=["accuracy"]
#    )

In [ ]:
#finetune_model()

In [ ]:
predictions = np.argmax(model.predict(validation_generator,verbose=1),axis=1)

In [ ]:
X_validation_df["predictions"]=predictions

In [ ]:
X_validation_df = X_validation_df.astype({"label":int})

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
X_validation_df["label_description"].hist(xrot=90,figsize=(12,9))
plt.tight_layout()

In [ ]:
accuracy_score(X_validation_df["label"],X_validation_df["predictions"])

In [ ]:
report=classification_report(X_validation_df["label"],X_validation_df["predictions"],target_names=["Cassava Bacterial Blight (CBB)","Cassava Brown Streak Disease (CBSD)","Cassava Green Mottle (CGM)","Cassava Mosaic Disease (CMD)","Healthy"])

print(report)                                         

In [ ]:
#print(classification_report(X_validation_df["label"],X_validation_df["predictions"]))

------------------------------------------- GRADCAM ------------------------------------

In [ ]:
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
last_conv_layer_name = "top_activation"

In [ ]:
classifier_layer_names = [
    "avg_pool",
    "BN_out1",
    "dropout_layer",
    "output_prediction"    
]

In [ ]:
image_test_path = TRAINING_IMAGES_PATH+X_validation_df['image_id'].values[4]

In [ ]:
display(Image(image_test_path))

In [ ]:
image_test_array = get_img_array(image_test_path,size=(512,512))

In [ ]:
preds = model.predict(image_test_array)

In [ ]:
print(preds)

In [ ]:
heatmap = make_gradcam_heatmap(image_test_array, model, last_conv_layer_name, classifier_layer_names)

In [ ]:
t = np.expand_dims(heatmap, axis=-1)

In [ ]:
m = np.zeros((16,16,3))

In [ ]:
m[:,:,0]=heatmap

In [ ]:
m.shape

In [ ]:
m_heatmap = keras.preprocessing.image.array_to_img(m)
mj_heatmap = m_heatmap.resize((512,512))

In [ ]:
mj_heatmap

In [ ]:
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    #jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

In [ ]:
def super_imposed_image(img_path,heatmap,save_path="superimposed.jpg"):
    
    # We load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)
    m = np.zeros((16,16,3))
    m[:,:,0]=heatmap
    # We rescale heatmap to a range 0-255
    #heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    #jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    #jet_colors = jet(np.arange(256))[:, :3]
    #jet_heatmap = jet_colors[heatmap]
    #print(jet_heatmap)
    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(m)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap + 0.6*img
    #superimposed_img = np.maximum(jet_heatmap,img)
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    #save_path = "elephant_cam.jpg"
    superimposed_img.save(save_path)

    # Display Grad CAM
    display(Image(save_path))
    return superimposed_img

In [ ]:
augmented_image = super_imposed_image(image_test_path,heatmap)

In [ ]:
augmented_image